In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from os import listdir

In [10]:
#Calculating and saving returns for various years#

data_path = "/home/harshit/Project/2014/" #Daily closing prices data
return_path = "/home/harshit/Project/Returns/2014/"
risk_free_ret_path = "/home/harshit/Project/risk_free_ret/2014/"
lis = pd.DataFrame(listdir(data_path),columns = ["Symbol"])  #A list of all data files in a year
factors= pd.read_csv("/home/harshit/Project/allfactors.csv")
risk_free = factors[["Date","Rf %"]]
for i in (lis["Symbol"]):
    if i == "renyi_entropy.csv": continue
    if i == "entropy.csv": continue
    df2 = pd.DataFrame(columns=["Date","Return"])
    data = pd.read_csv(data_path+i)
    company = i.split('.')[0]
    for j in range(len(data)):
            date = data.iloc[j]["Date"]
            previous = data.iloc[j-1][company]
            new = data.iloc[j][company]
            ret = ((previous-new)/previous)*100
            df = pd.DataFrame([[date,ret]],columns=["Date","Return"])
            df2 = df2.append(df,ignore_index=True)   # A DF having return data for whole year
    df2.at[0, 'Return'] = 0
    df2.Date = df2.Date.apply(lambda x: x.replace("-",""))
    df2["Date"] = df2["Date"].apply(pd.to_numeric)
    df = pd.merge(df2,risk_free,on="Date")
    df["R-Rf"] = df["Return"]-df["Rf %"]          # A data_frame having risk free return for the whole year
    df=df.drop(["Return","Rf %"],axis=1)
    df.rename(columns={"R-Rf":'Return'}, inplace=True)
    df.to_csv(risk_free_ret_path+company+".csv",index=False)
#    df2.to_csv(return_path+company+".csv",index=False)

In [56]:
# Making single file for return of all companies   
#Making Dataframe of list of companies
ret_path = "/home/harshit/Project/Returns/2013/"
rf_ret_path = "/home/harshit/Project/risk_free_ret/2013/" #risk free return path
file = listdir(ret_path)
rf_file = listdir(rf_ret_path)
file = pd.DataFrame(file,columns=["Name"])
rf_file = pd.DataFrame(rf_file,columns=["Name"])
#Combining to make single csv for return of all companies in a year
combined = pd.read_csv(ret_path+file.iloc[0]["Name"])
rf_combined = pd.read_csv(rf_ret_path+file.iloc[0]["Name"])
name = file.iloc[0]["Name"]
rf_name = rf_file.iloc[0]["Name"]
#combined=combined.drop(columns=["Unnamed: 0"])
combined.rename(columns={'Return':name}, inplace=True)
rf_combined.rename(columns={'Return':rf_name}, inplace=True)
#combined["Date"] = pd.to_datetime(combined["Date"])
for i in range(len(file)):
    if i==0:
        continue
    name = file.iloc[i]["Name"]
    if any([name== 'continuous_entropy.csv',name=='continuous_renyi_entropy.csv',name=='entropy.csv',
            name=='renyi_entropy.csv']):
        continue
    data = pd.read_csv(ret_path+name)
   # data["Date"] = pd.to_datetime(data["Date"])
    data.rename(columns={'Return':name}, inplace=True)
    combined = pd.merge_asof(combined,data,on='Date')

for i in range(len(file)):
    if i==0:
        continue
    name = file.iloc[i]["Name"]
    if any([name== 'continuous_entropy.csv',name=='continuous_renyi_entropy.csv',name=='entropy.csv',
            name=='renyi_entropy.csv']):
        continue
    data = pd.read_csv(rf_ret_path+name)
   # data["Date"] = pd.to_datetime(data["Date"])
    data.rename(columns={'Return':name}, inplace=True)
    rf_combined = pd.merge_asof(combined,data,on='Date')
combined
rf_combined
combined_path = "/home/harshit/Project/Returns/combined/"
rf_combined_path = "/home/harshit/Project/risk_free_ret/combined/"
combined.to_csv(combined_path+"combined2013.csv",index=False)
rf_combined.to_csv(rf_combined_path+"rf_combined2013.csv",index=False)

In [53]:
##################################################################################################################

#Finding average returns of top and bottom 30 companies
previous_entropy = pd.read_csv("/home/harshit/Project/2014/entropy.csv") #reading previous_entropy list to select 30
return_path = "/home/harshit/Project/risk_free_ret/combined/"
returns = pd.read_csv(return_path+"rf_combined2015.csv")
date = pd.DataFrame(pd.to_datetime(returns["Date"])) 
ret_ent = date
for i in previous_entropy["Symbol"]:
    name = i+".csv"
    if name in returns:                   # taking return of all firms present in entropy list and in next year
        ret = pd.DataFrame(returns[i+".csv"])
        ret = date.merge(ret,left_index=True, right_index=True)
        ret_ent = pd.merge_asof(ret_ent,ret,on="Date")
# Selecting firms with highest and lowest entropies
high_entropy = ret_ent.iloc[:,0:31]
low_entropy = ret_ent.iloc[:,-30:]
low_entropy = pd.merge(date,low_entropy,left_index=True,right_index=True)

#Finding mean of low and high entropy firm returns
high_mean = pd.DataFrame(high_entropy.mean(axis=1),columns=["high"])
low_mean = pd.DataFrame(low_entropy.mean(axis=1),columns=["low"])
avg= pd.merge(date,high_mean,right_index=True,left_index=True).merge(low_mean,right_index=True,left_index=True)

#Finding difference of high and low entropy averages
diff = pd.DataFrame(avg["high"]-avg["low"],columns=["high-low"]) 
avg = pd.merge(avg,diff,right_index=True,left_index=True)
avg_path = "/home/harshit/Project/risk_free_ret/averages/"
avg.to_csv(avg_path+"average2015.csv",index=False)

In [60]:
##################################################################################################################
#Same using renyi_entropy
#Finding average returns of top and bottom 30 companies
previous_entropy = pd.read_csv("/home/harshit/Project/2012/renyi_entropy.csv") #reading previous_entropy list to select 30
return_path = "/home/harshit/Project/risk_free_ret/combined/"
returns = pd.read_csv(return_path+"rf_combined2013.csv")
date = pd.DataFrame(pd.to_datetime(returns["Date"])) 
ret_ent = date
for i in previous_entropy["Symbol"]:
    name = i+".csv"
    if name in returns:                   # taking return of all firms present in entropy list and in next year
        ret = pd.DataFrame(returns[i+".csv"])
        ret = date.merge(ret,left_index=True, right_index=True)
        ret_ent = pd.merge_asof(ret_ent,ret,on="Date")
# Selecting firms with highest and lowest entropies
high_entropy = ret_ent.iloc[:,0:31]
low_entropy = ret_ent.iloc[:,-30:]
low_entropy = pd.merge(date,low_entropy,left_index=True,right_index=True)

#Finding mean of low and high entropy firm returns
high_mean = pd.DataFrame(high_entropy.mean(axis=1),columns=["high"])
low_mean = pd.DataFrame(low_entropy.mean(axis=1),columns=["low"])
avg= pd.merge(date,high_mean,right_index=True,left_index=True).merge(low_mean,right_index=True,left_index=True)

#Finding difference of high and low entropy averages
diff = pd.DataFrame(avg["high"]-avg["low"],columns=["high-low"]) 
avg = pd.merge(avg,diff,right_index=True,left_index=True)
avg_path = "/home/harshit/Project/risk_free_ret/averages/"
avg.to_csv(avg_path+"renyi_average2013.csv",index=False)

In [6]:
df2

,Date,Return
0,2017-01-02,40.299716
1,2017-01-03,-0.966757
2,2017-01-04,-2.110980
3,2017-01-05,-0.334503
4,2017-01-06,-1.420998
5,2017-01-09,-0.177830
6,2017-01-10,-1.070468
7,2017-01-11,-2.804833
8,2017-01-12,2.402154
9,2017-01-13,-1.347337
